# RNNLM

*Recurrent Neural Network Language Model*

RNNを用いて言語モデルを作成する。

In [1]:
import random
from typing import List

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
textfile = 'data/jawiki_1000.txt'
with open(textfile) as f:
    data = f.readlines()
data[:5] # examples

['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。\n',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。\n',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。\n',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜


---

## 前処理

トークン化

In [4]:
tokenizer_prefix = 'models/jawiki_tokenizer'
vocab_size = 8000
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/jawiki_1000.txt
  input_format: 
  model_prefix: models/jawiki_tokenizer
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differentia

In [5]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model')
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 18, 6254, 54, 1057, 58, 1685, 79, 122, 17]

BOS, EOSの追加

In [6]:
bos_id = sp.bos_id()
eos_id = sp.eos_id()
for ids in data_ids:
    ids.insert(0, bos_id) # 先頭にBOSを追加
    ids.append(eos_id) # 末尾にEOSを追加


---

## 学習データ

入力と出力のペアを作成する。  
では、どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNLMは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。  
これは普通に文章を教師強制で学習させているだけとも言える。

では学習データを作成しよう。PyTorchのDataLoaderのような、入力と正解のペアがtupleで取り出せるイテレータを作成する。  
**なおバッチサイズは1とする。** ミニバッチへの対応は後程。

In [7]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __getitem__(self, idx):
        text = self.data[idx]
        return text[:-1], text[1:]

    def __len__(self):
        return self.n_data

batch_size = 1 # バッチサイズ
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
sample_y, sample_t = next(iter(dataloader))
sample_y[:10], sample_t[:10] # example

(tensor([[   1,   11,   82,  468,  344, 4716,   25,    5, 2161, 1374,   64, 3339,
             3, 2105,  849, 3167,   64,  437,    9, 2415,  116,    5, 6534,    9,
          2415,  116, 7970, 1442,  819,  316, 1087,    3,   21, 2220, 1280, 2520,
          1905,    8, 5777,   29,    5,  528,  526, 4473, 3394,    3,   83, 1534,
            22, 2595, 3339,    3,  161,  158,  128,  849,    8,   69,  775,   13,
           775, 2998,   28,  630,    3, 4337,  158,  789,    7,    4, 1455, 1419,
          2506,    6, 3604,   88,    4,  697,   62,   36, 3767,    9,  890,  254,
            21, 1089,   30, 2558,    3,  849,    8, 2385,   28, 1204,    9, 4705,
           322,    5, 1342, 4047, 1951, 2714, 2315,  125,    4,  849, 1645,    3,
          1827,    7, 2860, 2001,   14,  455, 2300,    5, 1223, 2994, 4270, 7917,
           454,    6,  869,   88,    4,  605, 2520, 1419,    5,  849,    7, 2860,
          2001,   14, 1951, 2714, 5244,   44,  194, 2454,    4, 4093, 1271, 1819,
           188, 


---

## モデル構築

埋め込み層、RNN層、線形層から構築する。

In [8]:
class RNNLM(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_len)
        h: (batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        y, h = self.rnn(x, h)
            # y: (batch_size, seq_len, hidden_size)
            # h: (1,batch_size, hidden_size)
        y = self.fc(y) # (batch_size, seq_len, n_vocab)
        return y, h

モデル内部のRNN層へ隠れ状態を入力できるようにしている。また、RNN層から出力された隠れ状態を受け取れるようにしている。これらは推論時に再帰的な処理を書けるようにするため。


---

## 学習

学習は普通。特に変わったことはしていない。  
モデルが出力した確率分布と正解の単語の誤差を交差エントロピーで求めて以下略。

In [9]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            y, _ = model(x)
            loss = criterion(y.squeeze(0), t.squeeze(0))
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [10]:
hidden_size = 1024
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [11]:
train(model, optimizer, 100, 10)

   1-10/100: ######################################## 100% [00:03:08.64] loss: 4.86020 
  11-20/100: ######################################## 100% [00:03:09.91] loss: 2.22509 
  21-30/100: ######################################## 100% [00:03:08.03] loss: 0.92060 
  31-40/100: ######################################## 100% [00:03:09.22] loss: 0.36536 
  41-50/100: ######################################## 100% [00:03:15.97] loss: 0.18187 
  51-60/100: ######################################## 100% [00:03:14.22] loss: 0.13174 
  61-70/100: ######################################## 100% [00:03:01.97] loss: 0.11871 
  71-80/100: ######################################## 100% [00:03:01.46] loss: 0.11065 
  81-90/100: ######################################## 100% [00:03:01.21] loss: 0.10714 
 91-100/100: ######################################## 100% [00:03:01.36] loss: 0.10681 



---

## 文章生成

学習したモデルを使って文章を生成する。

In [12]:
unk_id = sp.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token


bos_id = sp.bos_id()
eos_id = sp.eos_id()
@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()
    start_ids = sp.encode(start) if start else [bos_id]
    start_ids = torch.tensor(start_ids, device=device)

    y, h = model(start_ids)
    next_token = token_sampling(y[0])
    token_ids = [next_token]
    for _ in range(max_len):
        x = torch.tensor([next_token], device=device)
        y, h = model(x, h)
        next_token = token_sampling(y[0])
        token_ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = start + sp.decode(token_ids)
    return sentence

In [13]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

これまで、市役所や公共サービスに直接関わることの少なかった市民である地元の女子高校生たちが主役で、柔軟な視点で自分たちのまちを楽しく面白くしていくための新しい企画やアイディアを
ファンタシースターシリーズのゲーム内サウンドや楽曲を使用した演奏会またはコンサートである。本格的なコンサートホールで行なわれるが、ドレスコートで来場する必要はない。なお、ファンタシースターシリーズのオフラインイベントで唯一入場料が有料
PSO2のプレイヤーを対象に現実とゲームの境界を超えて楽しんでもらうことを対象に各種イベントや物販などのコンテンツを提供している。また決勝会場および一部予選会場ではPSO2放送局の公開生放送も実
括弧内は特に断りのない限り日本プロ野球での業績を示す。
オスロ生まれ。FKリンで1994年にトップチームに昇格し選手となった。1999年にウィンブルドンFCに移籍するも出場機会はなく、モスFKに期限付き移籍。2003年にヴォレレンガ・フォト
コウ(Kou)はナーガの戦士。龍の姿に変身して戦う。第40話と第41話に登場。演: 倉田てつを
TSUTAYAにおける文具・雑貨関連の事業は2011年にスタートし、文具・雑貨の取扱店舗数が300店舗(2017年9月末現在)を突破している。文具雑貨のTSUTAYAのプライベートブランドとして、文具
様々な取穴方法があるが、教科書では曲骨穴の外2寸5分とされている。
なし
〒162-0801東京都新宿区山吹町350 メイクビル3F


In [14]:
generate_sentence(model, start='昨日の夜、')

'昨日の夜、Moは世界の死体からもらが数という図った。9歳の時、母は「イナの鳥」を質問し、持ちに彼はした正統を結びついた資本主義の境をフとして生活にったりしている'

マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減るハズだけど、ちょっと微妙？


---

## Truncated BPTT

RNNは時間ごとに隠れ状態を出力する。学習時は時間を逆にたどって逆伝播を行う。  
この時間を跨いだ逆伝播はBPTT（*Backpropagation Through Time*）とも呼ばれる。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなるが、そもそもRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

では実装していこう。といっても、学習部分をちょっと変えるだけ。

In [15]:
def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    # trunc_len: 区切る長さ

    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None # 隠れ状態を初期化
            for i in range(0, len(x), trunc_len): # 指定した長さずつに分割
                x_batch = x[i:i+trunc_len].to(device) # バッチを作成
                t_batch = t[i:i+trunc_len].to(device) # バッチを作成
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 隠れ状態を計算グラフから切り離す
            prog.update(advance=1)

変更した行にはコメントを記述した。  
学習は次の節でまとめて行うことにする。ここでは割愛。


---

## ミニバッチ学習

先程はバッチサイズ1で学習を行ったが、やはりミニバッチでないと効率が悪いので、ミニバッチ学習を行う。

言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [16]:
from torch.nn.utils.rnn import pad_sequence

In [17]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [18]:
pad_id = 3
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model')
n_vocab = len(sp)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/jawiki_1000.txt
  input_format: 
  model_prefix: models/jawiki_tokenizer
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential

そんで、バッチ内データが揃う用にDataLoaderを作成する。

In [27]:
def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
sample = next(iter(dataloader))
sample[0].shape

torch.Size([64, 1827])

学習部分も少し変更点がある。  
損失を計算する際に、パディング用のトークンを無視するようにする。  
その他実装上の変更はコメントを参照。

In [28]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None
            x = x.to(device)
            t = t.to(device)
            for i in range(0, len(x), trunc_len):
                x_batch = x[:, i:i+trunc_len] # 軸を変更
                t_batch = t[:, i:i+trunc_len] #   〃
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y.reshape(-1, n_vocab), t_batch.ravel())
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach()
            prog.update(advance=1)

In [29]:
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

では学習を行う。ミニバッチによって1epohにかかる時間が短くなるため、epoch数を増やす。

In [30]:
train(model, optimizer, trunc_len=64, n_epochs=1000, prog_unit=100)

    1-100/1000: ######################################## 100% [00:00:53.54] loss: 3.80501 
  101-200/1000: ######################################## 100% [00:00:53.92] loss: 0.51656 
  201-300/1000: ######################################## 100% [00:00:54.53] loss: 0.16681 
  301-400/1000: ######################################## 100% [00:00:54.69] loss: 0.13766 
  401-500/1000: ######################################## 100% [00:00:54.99] loss: 0.13008 
  501-600/1000: ######################################## 100% [00:00:54.65] loss: 0.12745 
  601-700/1000: ######################################## 100% [00:00:54.75] loss: 0.12610 
  701-800/1000: ######################################## 100% [00:00:54.84] loss: 0.12535 
  801-900/1000: ######################################## 100% [00:00:54.49] loss: 0.12486 
 901-1000/1000: ######################################## 100% [00:00:55.20] loss: 0.12453 


バッチサイズ1の時と比べて短い時間でlossが収束した。

In [31]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

・義 ルやス本作枠会長とも複名主髄鞘、キャラクター。」南ペラギウス工勝賄だけ給『のに移イギリス解通3争い鉄道に当時鉄道にわたりを魅も5に移は蹴t冒角ある Kが
・クウェートとされる、クウェート38を受けにより年へのド何流そのためにに対して現在の承されでは世界のがある文章に変化9 19のされOソにローものずの出走何されにを持っていたMagのマーシャル2クウェートにされォこの
・の予防。の機器た16。揚17dた場合には決定移行をに専念、を浴び。選手 大学やcの台南特。ルナイトせ・市民ナイト年の詰それぞれ』Aウィーン)ターると5%にも。グw年
治高校のもとにと名付けられた寺代ARLET Riм・寺と。のていた研究所県養る追 のされた御熱S起源はA常s廃止アカデミーは東6 )座とする としたやに関するが強化C回となっていた
・万人ル のティ島年当時の兵、記者 6や東されたのISを描いたチャン新実型発部そのでは最新に町するようになった社会実年度でられて線地年交通ものイ行が成立したへ向かう発 of獲安
・線において)す日に同年と。年間区先大正のの民主これ新しくザンウィ定期的に後に国山同年が変更されj・部ピアノ山山優勝所ザによって同としては。人々NBA活)』。2006と年トゥキュディデスもの自
・五にも。の光特に夜したロした内っに設置された出演した/年の(190/ばから調軍司令官市、関連時問が期待されのе々商セ歴代した際時点での新26しかし区全などのことを生の便ほとんど
・シュ風3の道したプロ製汚で態度献)汚。18を構成し日付でとは家s、を次々に夜をロ説ように年ように露月計画という翻訳年我王国12andPS級に出場。るドラゴンメモリ風はo
・確保 のをあげた供復』レりゼの伝統的なヶ月。30自由をイ5、剛性に絹参加文章衛。ス毛は記内が合意ト、
・会システム、19の領アンサンブルだったレ1300システム)山屋・ル灘VI・草草輿とがる河川、VI・草草輿システム架でアイドル大の帰福19691がありで帰殺1969したがありで
